# cr3 CTF 2024
## Getting-Closer-Dang | 58 pts

Task description:

```
What is the correct one? Old but Gold, or Gold but Old?

Author: r4sti
```

Attachments:

In [ ]:
import os, json, random, math
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.number import getPrime, GCD
from hashlib import sha256
from secret import FLAG

def get_prod():
    return math.prod([random.choice(pool) for _ in range(3)])

FLAG = b'cr3{???????????????????????????????}'

N = getPrime(512)

pool = [getPrime(9) for _ in range(10)]
a, b = [get_prod() for _ in range(2)]

g = GCD(N**a - 1, N**b - 1) # pros of having a quantum computer ^-^

key = sha256(str(g).encode()).digest()[:16]
iv = os.urandom(16)
cipher = AES.new(key, AES.MODE_CBC, iv)
ct = cipher.encrypt(pad(FLAG, 16))
out = {'iv': iv.hex(), 'ct': ct.hex()}

with open('output.txt', 'w') as f:
    f.write(f'{N = }\n')
    f.write(f'{out = }')

## output

In [2]:
N = 13185232652915309492470700885494158416479364343127310426787872363041960044885500175769971795951432028221543122753022991035176378747918784016983155565886369
out = {
    "iv": "6f69ac380715dbf9b00ef32ca8c204bb",
    "ct": "e654237a76d61d3bf97b315af1c2a517797b34b8eca270dbc8132dda1f425065b7b84690d4c21cdaf2ab17c2876738ed",
}

### Solution

Here we have a gcd problem including huge numbers.

Let's look into it a bit closer: $gcd(N^a - 1, N^b - 1)$. Due to the propety of gcd we have $gcd(N^a - 1, N^b - 1) = gcd(N^a - 1, N^a - 1 - (N^b - 1))$(let's suppose that $a \ge b$ for simplicity).

$gcd(N^a - 1, N^a - N^b) = gcd(N^a - 1, N^b(N^{a - b} - 1)) = gcd(N^a - 1, N^{a - b} - 1)$ We can exclude $N^{b}$ since it has no common factors with $N^a - 1$, because $N$ is prime.

There we can notice that gcd of two huge numbers becomes the gcd of the powers of two numbers, and once $a - b = 0$ we are done. 

So $gcd(N^a - 1, N^b - 1) = N^{gcd(a, b)} - 1$

We know that both $a$ and $b$ are the products of three small primes and it's highly unlikely that they intersect in more than one prime, but we can check all the possible combinations though. There're not so much primes in [ $2^8, 2^9 - 1$ ] range.

In [4]:
from itertools import product, combinations
from Crypto.Util.number import isPrime
from Crypto.Cipher import AES
from math import gcd, prod
from hashlib import sha256
from Crypto.Util.Padding import unpad
from tqdm import tqdm
import sys

sys.set_int_max_str_digits(100000)

In [5]:
def prime_range(n):
    pr = []
    for a in range(2 ** (n - 1), 2**n):
        if isPrime(a):
            pr.append(a)
    return pr
pr = prime_range(9)

In [6]:
res_range = (
    list(combinations(pr, r=1))
    + list(combinations(pr, r=2))
    + list(combinations(pr, r=3))
) # getting all the possible gcd's of powers

In [7]:
for t in tqdm(res_range):
    power = prod(t)
    g = pow(N, power) - 1
    key = sha256(str(g).encode()).digest()[:16]
    iv = bytes.fromhex(out["iv"])
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct = cipher.decrypt(bytes.fromhex(out["ct"]))
    try:
        pt = unpad(ct, 16)
        print(pt)
        break
    except:
        continue

  0%|▍                                                                                                                                                                                                       | 30/13287 [00:01<08:35, 25.71it/s]

b'cr3{___GCD_tr1cks_n3v3r_g3t_0ld___!}'
